In [1]:
import requests
import json

In [3]:
class Embedder:
    def __init__(self, model_name):
        # Updated base URL to the new endpoint
        base_url = "http://ray.ml.uat.us.workloads.elizacloud.com/serve/"
        
        if model_name == "e5_mistral_embed_384":
            self.url = base_url + "e5embed384"
        else:
            raise ValueError(f"Unsupported model name: {model_name}")

        self.headers = {"Content-Type": "application/json"}
        print(f"[INFO] Initialized Embedder with URL: {self.url}")

    def get_embeddings(self, texts):
        data = {"texts": texts}
        print(f"[DEBUG] Sending data: {data} to {self.url}")
        
        try:
            response = requests.post(self.url, headers=self.headers, json=data, verify=False)
            print(f"[DEBUG] Received response with status code: {response.status_code}")
            response.raise_for_status()

            response_json = response.json()
            print("[INFO] API Response:", response_json)

            # Check for embedding field in the response
            if "dense_embeddings" in response_json:
                embeddings = response_json["dense_embeddings"]
                print("[INFO] Found dense_embeddings directly in response.")
                return embeddings
            elif "result" in response_json:
                # Try to convert the result if it's a string
                if isinstance(response_json["result"], str):
                    print("[DEBUG] Parsing 'result' string into JSON...")
                    response_json["result"] = json.loads(response_json["result"])
                    
                if "dense_embeddings" in response_json["result"]:
                    embeddings = response_json["result"]["dense_embeddings"]
                    print("[INFO] Found dense_embeddings inside result.")
                    return embeddings
                
            raise ValueError(f"Unexpected API response format: {response_json}")
        except requests.exceptions.RequestException as e:
            print(f"[ERROR] API Request Error: {e}")
            return None
        except ValueError as e:
            print(f"[ERROR] Response Parsing Error: {e}")
            return None

In [ ]:
e5_embedder = Embedder("e5_mistral_embed_384")

test_sentences = [
    "Hello, world!"
]
print(f"[INFO] Requesting embeddings for sentences: {test_sentences}")

# Get the embeddings
embedding_vectors = e5_embedder.get_embeddings(test_sentences)

if embedding_vectors is not None:
    print("Embedding Vectors:")
    print(json.dumps(embedding_vectors, indent=2))
else:
    print("Unable to retrieve embedding vectors.")

In [ ]:
# Assuming chunk_text and embed_model.get_embeddings(text) exist

texts = []
keys = []
metadata_list = []

for chunk in chunks:  # chunks being original large text chunks or documents
    text = chunk["metadata"]["text"]
    s3_path = chunk.get("s3_path", "unknown_path")  # or wherever you store path info

    text_chunks = chunk_text(text, s3_path, token_limit=500, tokenizer_name="bert-base-uncased")
    
    for i, text_chunk in enumerate(text_chunks):
        texts.append(text_chunk)
        
        # Create a unique key per chunk, e.g. original_key + chunk index
        keys.append(f"{chunk['key']}_chunk{i}")
        
        # Copy & update metadata for this chunk if needed
        meta = chunk["metadata"].copy()
        meta["chunk_index"] = i
        metadata_list.append(meta)

vectors = []
for i, text in enumerate(texts):
    embedding = embed_model.get_embeddings(text)  # single chunk at a time
    vectors.append({
        "key": keys[i],
        "data": {"float32": embedding},
        "metadata": metadata_list[i]
    })


In [2]:
EMBEDINNGS_URL = "https://zgggzg2iqg.execute-api.us-east-1.amazonaws.com/dev/get_embeddings"
API_KEY = "2jIpWCyNRg3Y8lkbmWG0tkyXwYlJn5QaZ1F3yKf7"

def get_text_embedding(texts, model= 'e5_mistral_embed_384'):

    payload = {
        "model_name": model,
        "texts": texts
    }

    headers = {
        "x-api-key": API_KEY
    }

    response = requests.post(EMBEDINNGS_URL, json=payload, headers=headers)
    response.raise_for_status()

    raw_body = response.json().get('body')

    parsed_body = json.loads(raw_body)

    embeddings_data = parsed_body.get('embeddings')

    if not embeddings_data:
        raise KeyError("No embedding data found in response.")

    return embeddings_data

print(get_text_embedding(['Taher Hellot']))

TypeError: the JSON object must be str, bytes or bytearray, not NoneType

In [4]:
import os
import json
import requests
import numpy as np

def _extract_embeddings_obj(obj):

    if isinstance(obj, dict) and "embeddings" in obj:
        return obj["embeddings"]

    # Case B: wrapper with body string
    if isinstance(obj, dict) and "body" in obj:
        try:
            body = obj["body"]
            if isinstance(body, str):
                inner = json.loads(body)
            else:
                inner = body
            if isinstance(inner, dict) and "embeddings" in inner:
                return inner["embeddings"]
        except Exception:
            pass

    raise KeyError("No 'embeddings' found in response object")

def get_text_embedding(texts, model='e5_mistral_embed_384', timeout=8):
    if isinstance(texts, str):
        texts = [texts]
    if not isinstance(texts, list) or not texts:
        raise ValueError("Input 'texts' must be a non-empty list of strings.")

    headers = {
        "x-api-key": API_KEY,
        "Content-Type": "application/json"
    }

    out = []
    for text in texts:
        if not isinstance(text, str):
            raise ValueError("Each item in 'texts' must be a string.")

        payload = {"model_name": model, "texts": [text]}

        try:
            resp = requests.post(EMBEDINNGS_URL, json=payload, headers=headers, timeout=timeout)
            resp.raise_for_status()

            # Try both shapes
            obj = resp.json()
            embeddings = _extract_embeddings_obj(obj)

            if (not isinstance(embeddings, list)) or len(embeddings) != 1 or (not isinstance(embeddings[0], list)):
                raise KeyError("Response did not contain a single embedding vector")

            # Convert to float32 for consistency / memory
            vec = np.array(embeddings[0], dtype=np.float32).tolist()
            out.append(vec)

        except Exception as e:
            # Print useful diagnostics once
            print(f"[ERROR] Failed to get embedding for '{text}': {e}")
            try:
                print(f"[DEBUG] HTTP {resp.status_code} body: {resp.text[:500]}")
            except Exception:
                pass
            out.append(None)

    return out[0] if len(out) == 1 else out

print(get_text_embedding(['Taher Hellot']))


[-0.07986614853143692, 0.0891704335808754, -0.010766401886940002, 0.021151578053832054, -0.033024583011865616, 0.049935173243284225, 0.06037098169326782, -0.07330954074859619, 0.012036563828587532, 0.028119603171944618, 0.04819347709417343, -0.01807309128344059, -0.018687458708882332, 0.02039937861263752, 0.010969646275043488, -0.09223239868879318, -0.0008965861052274704, 0.07051318138837814, -0.11643633991479874, 0.032519880682229996, 0.07876076549291611, -0.056114163249731064, 0.015588684007525444, -0.04337730631232262, 0.005865859799087048, -0.02530551888048649, 0.02141365222632885, 0.007881022989749908, -0.09419673681259155, -0.16596245765686035, -0.03643881902098656, 0.0020525227300822735, 0.025138087570667267, -0.017854861915111542, 0.06110985204577446, -0.05482398346066475, -0.01196559239178896, -0.008604083210229874, -0.006771320477128029, 0.061452947556972504, -0.005728378891944885, -0.03197328746318817, 0.01860843412578106, -0.06509000808000565, -0.04387306794524193, -0.01240